In [ ]:
# Detecting Morse Code using Fingers
import cv2
import math
import time
import numpy as np
import mediapipe as mp

cap = cv2.VideoCapture(0)
mp_hands = mp.solutions.hands
mp_draw = mp.solutions.drawing_utils
hands = mp_hands.Hands(min_detection_confidence=0.8,min_tracking_confidence=0.8)

morse_code = ""
touch_start = None
space_shown = False

def distance(p1,p2):
    return math.hypot(p2[0]-p1[0],p2[1]-p1[1])



while cap.isOpened():
    r,f = cap.read()
    if r==False:
        break
    f = cv2.flip(f,1)
    rgb = cv2.cvtColor(f,cv2.COLOR_BGR2RGB)
    res = hands.process(rgb)
    h,w,_ = f.shape


    if res.multi_hand_landmarks:
        landmarks = res.multi_hand_landmarks[0]
        lm = landmarks.landmark
        index_tip = int(lm[8].x*w),int(lm[8].y*h)
        thumb_tip = int(lm[4].x*w),int(lm[4].y*h)

        d = distance(index_tip,thumb_tip)
        cv2.line(f,index_tip,thumb_tip,(255,0,255),1)
        cv2.putText(f,f'Distance: {d:.1f}',(30,50),cv2.FONT_HERSHEY_COMPLEX_SMALL,0.7,(255,255,255),2)

        if d<40:
            if touch_start is None:
                touch_start = time.time()
        else:
            if touch_start is not None:
                duration = time.time() - touch_start
                if duration>=0.2:
                    symbol='.' if duration<1 else '-'
                    morse_code+=symbol
                    # print(f"Touch Duration: {duration:.2f} -> {symbol}")
                touch_start = None
                space_shown = False
        mp_draw.draw_landmarks(f,landmarks,mp_hands.HAND_CONNECTIONS)
    else:
        if not space_shown:
            morse_code+=' '
            # print("[SPACE]")
            space_shown=True
        touch_start=None
    cv2.putText(f,f"Morse Code : {morse_code}",(10,450),cv2.FONT_HERSHEY_COMPLEX_SMALL,1,(0,255,255),2)
    cv2.imshow("Sign Language Detector",f)
    if cv2.waitKey(25) & 0xff == ord('d'):
        break

cap.release()
cv2.destroyAllWindows()



# Morse Code Decoder
MORSE_DICT = {
    '.-': 'A', '-...': 'B', '-.-.': 'C',
    '-..': 'D', '.': 'E', '..-.': 'F',
    '--.': 'G', '....': 'H', '..': 'I',
    '.---': 'J', '-.-': 'K', '.-..': 'L',
    '--': 'M', '-.': 'N', '---': 'O',
    '.--.': 'P', '--.-': 'Q', '.-.': 'R',
    '...': 'S', '-': 'T', '..-': 'U',
    '...-': 'V', '.--': 'W', '-..-': 'X',
    '-.--': 'Y', '--..': 'Z',
    '-----': '0', '.----': '1', '..---': '2',
    '...--': '3', '....-': '4', '.....': '5',
    '-....': '6', '--...': '7', '---..': '8',
    '----.': '9'
}

def decode_morse_code(morse_code):
    words = morse_code.strip().split(' ')
    decoded = []
    decoded_word = ''.join([MORSE_DICT.get(c,'?') for c in words if c])
    decoded.append(decoded_word)
    return ' '.join(decoded)


print(decode_morse_code(morse_code))

In [ ]:
# Detecting Morse Code using Eye blink
import time
import math
import cv2
import numpy as np
import mediapipe as mp

cap = cv2.VideoCapture(0)
mp_face = mp.solutions.face_mesh
mp_draw = mp.solutions.drawing_utils
face = mp_face.FaceMesh(refine_landmarks=True)

LEFT_EYE = [263, 387, 385, 362, 380, 373]
RIGHT_EYE = [33, 160, 158, 133, 153, 144]

blink_start = None
morse_code = ""

last_blink_time = time.time()

def classify_blink(duration):
    return '.' if duration < 0.7 else '-'

def calculate_ear(eye_points,landmarks):
    p1 = landmarks[eye_points[0]]
    p2 = landmarks[eye_points[1]]
    p3 = landmarks[eye_points[2]]
    p4 = landmarks[eye_points[3]]
    p5 = landmarks[eye_points[4]]
    p6 = landmarks[eye_points[5]]

    # for idx in LEFT_EYE:
    #     print(f"{idx}: ({landmarks[idx].x:.3f}),({landmarks[idx].y:.3f})")

    vertical1 = math.dist((p2.x,p2.y),(p6.x,p6.y))
    vertical2 = math.dist((p3.x,p3.y),(p5.x,p5.y))

    horizontal = math.dist((p1.x,p1.y),(p4.x,p4.y))
    ear = (vertical1+vertical2)/(2.0*horizontal)
    return ear


while cap.isOpened():
    r,f = cap.read()
    if r==False:
        break
    f = cv2.flip(f,1)
    rgb = cv2.cvtColor(f,cv2.COLOR_BGR2RGB)
    h,w,_ = f.shape
    res = face.process(rgb)

    if res.multi_face_landmarks:
        # for landmarks in res.multi_face_landmarks:
        mesh = res.multi_face_landmarks[0]
        landmarks = mesh.landmark

        left_ear = calculate_ear(LEFT_EYE,landmarks)
        right_ear = calculate_ear(RIGHT_EYE,landmarks)
        ear = (left_ear+right_ear)/2
        cv2.putText(f,f'Ear: {ear:.2f}',(30,50),cv2.FONT_HERSHEY_COMPLEX_SMALL,1,(255,255,255),1)

        if ear<0.25:

            if blink_start is None:
                blink_start=time.time()
        else:
            if blink_start is not None:
                duration = time.time() - blink_start
                symbol = classify_blink(duration)
                morse_code+=symbol
                print(f'Blink Duration: {duration:.2f}->{symbol}')
                time_since_last = time.time()-last_blink_time
                if time_since_last>5:
                    morse_code+=' '
                    print("[SPACE]")
                last_blink_time=time.time()
                blink_start=None
        cv2.putText(f,f"Morse Code : {morse_code}",(30,100),cv2.FONT_HERSHEY_SIMPLEX,1,(0,255,255),2)


    cv2.imshow("Morse Code Decoder",f)
    if cv2.waitKey(25) & 0xff == ord('d'):
        break   


cap.release()
cv2.destroyAllWindows()